In [2]:

import pandas as pd

train_df = pd.read_csv('../data/imdb_train.csv', header=0)
train_df.head()

,text,label
0,This movie makes me want to throw up every tim...,0
1,Listening to the director's commentary confirm...,0
2,One of the best Tarzan films is also one of it...,1
3,Valentine is now one of my favorite slasher fi...,1
4,No mention if Ann Rivers Siddons adapted the m...,0


In [3]:
import pandas as pd

test_df = pd.read_csv('../data/imdb_test.csv', header=0)
test_df.head()

,text,label
0,What I hoped for (or even expected) was the we...,0
1,Garden State must rate amongst the most contri...,0
2,There is a lot wrong with this film. I will no...,1
3,"To qualify my use of ""realistic"" in the summar...",1
4,Dirty War is absolutely one of the best politi...,1


In [8]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Let's get a list of stop words from the NLTK library
stop = stopwords.words('english')

# These words are important for our problem. We don't want to remove them.
excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# New stop word list
stop_words = [word for word in stop if word not in excluding]

snow = SnowballStemmer('english')

def process_text(texts): 
    final_text_list=[]
    for sent in texts:
        
        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ""
            
        filtered_sentence=[]
        
        sent = sent.lower() # Lowercase 
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:
        
        for w in word_tokenize(sent):
            # We are applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stop words
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):  
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) #final string of cleaned words
 
        final_text_list.append(final_string)
        
    return final_text_list

In [9]:
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(train_df[["text"]],
                                                  train_df["label"],
                                                  test_size=0.10,
                                                  shuffle=True,
                                                  random_state=324
                                                 )

In [10]:
print("Processing the text fields")
train_text_list = process_text(X_train["text"].tolist())
val_text_list = process_text(X_val["text"].tolist())

Processing the reviewText fields


In [17]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier

### PIPELINE ###
##########################

pipeline = Pipeline([
    ('text_vect', CountVectorizer(binary=True,
                                  max_features=15)),
    ('knn', KNeighborsClassifier(n_neighbors=15))  
                                ])

# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=15)),
                ('knn', KNeighborsClassifier(n_neighbors=15))])

In [18]:
# We using lists of processed text fields 
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=15)),
                ('knn', KNeighborsClassifier(n_neighbors=15))])

In [19]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[686 594]
 [551 669]]
              precision    recall  f1-score   support

           0       0.55      0.54      0.55      1280
           1       0.53      0.55      0.54      1220

    accuracy                           0.54      2500
   macro avg       0.54      0.54      0.54      2500
weighted avg       0.54      0.54      0.54      2500

Accuracy (validation): 0.542


In [20]:
print("Processing the text fields")
test_text_list = process_text(test_df["text"].tolist())

Processing the text fields


In [21]:
pipeline.fit(test_text_list, test_df["label"].values)

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=15)),
                ('knn', KNeighborsClassifier(n_neighbors=15))])

In [22]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(test_text_list)
print(confusion_matrix(test_df["label"].values, val_predictions))
print(classification_report(test_df["label"].values, val_predictions))
print("Accuracy (validation):", accuracy_score(test_df["label"].values, val_predictions))

[[7834 4666]
 [4503 7997]]
              precision    recall  f1-score   support

           0       0.64      0.63      0.63     12500
           1       0.63      0.64      0.64     12500

    accuracy                           0.63     25000
   macro avg       0.63      0.63      0.63     25000
weighted avg       0.63      0.63      0.63     25000

Accuracy (validation): 0.63324
